## Importing necessary packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# preprocessing/decomposition
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA, FastICA, FactorAnalysis, KernelPCA

# keras 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint

# model evaluation
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# supportive models
from sklearn.ensemble import GradientBoostingRegressor
# feature selection (from supportive model)
from sklearn.feature_selection import SelectFromModel

# to make results reproducible
seed = 42 # was 42

In [2]:
# Read datasets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
# save IDs for submission
id_test = test['ID'].copy()

In [4]:
# glue datasets together
total = pd.concat([train, test], axis=0)
print('initial shape: {}'.format(total.shape))

# binary indexes for train/test set split
is_train = ~total.y.isnull()

initial shape: (8418, 378)


In [5]:
# find all categorical features
cf = total.select_dtypes(include=['object']).columns
print total[cf].head()

   X0 X1  X2 X3 X4 X5 X6 X8
0   k  v  at  a  d  u  j  o
1   k  t  av  e  d  y  l  o
2  az  w   n  c  d  x  j  x
3  az  t   n  f  d  x  l  e
4  az  v   n  f  d  h  d  n


In [6]:
# make one-hot-encoding convenient way - pandas.get_dummies(df) function
dummies = pd.get_dummies(
    total[cf],
    drop_first=False # you can set it = True to ommit multicollinearity (crucial for linear models)
)

print('oh-encoded shape: {}'.format(dummies.shape))
print dummies.head()

oh-encoded shape: (8418, 211)
   X0_a  X0_aa  X0_ab  X0_ac  X0_ad  X0_ae  X0_af  X0_ag  X0_ai  X0_aj  ...   \
0     0      0      0      0      0      0      0      0      0      0  ...    
1     0      0      0      0      0      0      0      0      0      0  ...    
2     0      0      0      0      0      0      0      0      0      0  ...    
3     0      0      0      0      0      0      0      0      0      0  ...    
4     0      0      0      0      0      0      0      0      0      0  ...    

   X8_p  X8_q  X8_r  X8_s  X8_t  X8_u  X8_v  X8_w  X8_x  X8_y  
0     0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     1     0  
3     0     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0     0  

[5 rows x 211 columns]


In [7]:
# get rid of old columns and append them encoded
total = pd.concat(
    [
        total.drop(cf, axis=1), # drop old
        dummies # append them one-hot-encoded
    ],
    axis=1 # column-wise
)

print('appended-encoded shape: {}'.format(total.shape))

appended-encoded shape: (8418, 581)


In [8]:
# recreate train/test again, now with dropped ID column
train, test = total[is_train].drop(['ID'], axis=1), total[~is_train].drop(['ID', 'y'], axis=1)

# drop redundant objects
del total

# check shape
print('\nTrain shape: {}\nTest shape: {}'.format(train.shape, test.shape))


Train shape: (4209, 580)
Test shape: (4209, 579)


In [9]:
# Calculate additional features: dimensionality reduction components
n_comp=10 # was 10

# uncomment to scale data before applying decompositions
# however, all features are binary (in [0,1] interval), i don't know if it's worth something
train_scaled = train.drop('y', axis=1).copy()
test_scaled = test.copy()
'''
ss = StandardScaler()
ss.fit(train.drop('y', axis=1))

train_scaled = ss.transform(train.drop('y', axis=1).copy())
test_scaled = ss.transform(test.copy())
'''

# PCA
pca = PCA(n_components=n_comp, random_state=seed)
pca2_results_train = pca.fit_transform(train_scaled)
pca2_results_test = pca.transform(test_scaled)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train_scaled)
ica2_results_test = ica.transform(test_scaled)

# Factor Analysis
fca = FactorAnalysis(n_components=n_comp, random_state=seed)
fca2_results_train = fca.fit_transform(train_scaled)
fca2_results_test = fca.transform(test_scaled)

# Append it to dataframes
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-1]
    test['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    train['ica_' + str(i)] = ica2_results_train[:,i-1]
    test['ica_' + str(i)] = ica2_results_test[:, i-1]
    
    #train['fca_' + str(i)] = fca2_results_train[:,i-1]
    #test['fca_' + str(i)] = fca2_results_test[:, i-1]
   

/home/krohitm/anaconda2/envs/krohitm/lib/python2.7/site-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [10]:
# create augmentation by feature importances as additional features
t = train['y']
tr = train.drop(['y'], axis=1)

# Tree-based estimators can be used to compute feature importances
clf = GradientBoostingRegressor(
                max_depth=4, 
                learning_rate=0.005, 
                random_state=seed, 
                subsample=0.95, 
                n_estimators=200
)

# fit regressor
clf.fit(tr, t)

# df to hold feature importances
features = pd.DataFrame()
features['feature'] = tr.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

print features.tail()

# select best features
model = SelectFromModel(clf, prefit=True)
train_reduced = model.transform(tr)

test_reduced = model.transform(test.copy())

# dataset augmentation
train = pd.concat([train, pd.DataFrame(train_reduced)], axis=1)
test = pd.concat([test, pd.DataFrame(test_reduced)], axis=1)

# check new shape
print('\nTrain shape: {}\nTest shape: {}'.format(train.shape, test.shape))

         importance
feature            
X119       0.041776
X118       0.044236
pca_6      0.091539
X315       0.128361
X314       0.610175

Train shape: (4209, 617)
Test shape: (4209, 616)


In [12]:
# define custom R2 metrics for Keras backend
from keras import backend as K

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [14]:
from keras.optimizers import SGD, Adagrad, Adadelta
# base model architecture definition
def model():
    model = Sequential()
    #input layer
    model.add(Dense(input_dims, input_dim=input_dims))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    # hidden layers
    model.add(Dense(input_dims))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//2))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//4, activation=act_func))
    
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  #optimizer='adam',
                  optimizer=Adadelta(),#SGD(lr=0.0001, momentum=0.9),
                  #optimizer=Adagrad(),
                  metrics=[r2_keras] # you can add several if needed
                 )
    
    # Visualize NN architecture
    print(model.summary())
    return model

In [15]:
# initialize input dimension
input_dims = train.shape[1]-1

#activation functions for hidden layers
act_func = 'tanh' # could be 'relu', 'sigmoid', ...

# make np.seed fixed
np.random.seed(seed)

# initialize estimator, wrap model in KerasRegressor
estimator = KerasRegressor(
    build_fn=model, 
    nb_epoch=100, 
    batch_size=10,
    verbose=1
)

In [16]:
# X, y preparation
X, y = train.drop('y', axis=1).values, train.y.values
print(X.shape)

# X_test preparation
X_test = test
print(X_test.shape)

# train/validation split
X_tr, X_val, y_tr, y_val = train_test_split(
    X, 
    y, 
    test_size=0.20, 
    random_state=seed
)

(4209, 616)
(4209, 616)


In [18]:
# define path to save model
import os
model_path = 'keras_model_adadelta.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=10, # was 10
        verbose=1),
    
    ModelCheckpoint(
        model_path, 
        monitor='val_loss', 
        save_best_only=True, 
        verbose=0)
]

# fit estimator
estimator.fit(
    X_tr, 
    y_tr, 
    #nb_epoch=10, # increase it to 20-100 to get better results
    validation_data=(X_val, y_val),
    verbose=2,
    callbacks=callbacks,
    shuffle=True
)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_6 (Dense)                  (None, 616)           380072      dense_input_2[0][0]              
____________________________________________________________________________________________________
batchnormalization_4 (BatchNorma (None, 616)           2464        dense_6[0][0]                    
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 616)           0           batchnormalization_4[0][0]       
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 616)           0           activation_4[0][0]               
___________________________________________________________________________________________

In [21]:
# if best iteration's model was saved then load and use it
if os.path.isfile(model_path):
    estimator = load_model(model_path, custom_objects={'r2_keras': r2_keras})

# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y_tr, estimator.predict(X_tr))**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y_tr, estimator.predict(X_tr)))) # R^2 train

# check performance on validation set
print('MSE val: {}'.format(mean_squared_error(y_val, estimator.predict(X_val))**0.5)) # mse val
print('R^2 val: {}'.format(r2_score(y_val, estimator.predict(X_val)))) # R^2 val
pass

MSE train: 7.80798918632
R^2 train: 0.623668164219
MSE val: 7.92366340175
R^2 val: 0.596630478318


### Temporary check for results

In [19]:
# if best iteration's model was saved then load and use it
if os.path.isfile(model_path):
    estimator = load_model(model_path, custom_objects={'r2_keras': r2_keras})

# check performance on train set
print('MSE train: {}'.format(mean_squared_error(y_tr, estimator.predict(X_tr))**0.5)) # mse train
print('R^2 train: {}'.format(r2_score(y_tr, estimator.predict(X_tr)))) # R^2 train

# check performance on validation set
print('MSE val: {}'.format(mean_squared_error(y_val, estimator.predict(X_val))**0.5)) # mse val
print('R^2 val: {}'.format(r2_score(y_val, estimator.predict(X_val)))) # R^2 val
pass

MSE train: 7.80798918632
R^2 train: 0.623668164219
MSE val: 7.92366340175
R^2 val: 0.596630478318


In [20]:
# predict results
res = estimator.predict(X_test.values).ravel()

# create df and convert it to csv
output = pd.DataFrame({'id': id_test, 'y': res})
output.to_csv('../results/adadelta.csv', index=False)

In [24]:
estimator.predict(X_tr).ravel()

array([ 91.89006805,  91.54103088,  93.6187439 , ...,  93.46170044,
        92.79412079,  95.83500671], dtype=float32)

# Trying another method

In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder

In [21]:
# read datasets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [22]:
from sklearn.decomposition import PCA, FastICA
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
ica2_results_test = ica.transform(test)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-1]
    test['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    train['ica_' + str(i)] = ica2_results_train[:,i-1]
    test['ica_' + str(i)] = ica2_results_test[:, i-1]
    
y_train = train["y"]
y_mean = np.mean(y_train)

In [23]:
 ()# mmm, xgboost, loved by everyone ^-^
import xgboost as xgb

# prepare dict of params for xgboost to run with
xgb_params = {
    'n_trees': 500, 
    'eta': 0.005,
    'max_depth': 4,
    'subsample': 0.95,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(train.drop('y', axis=1), y_train)
dtest = xgb.DMatrix(test)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=650, # increase to have better results (~700)
                   early_stopping_rounds=50,
                   verbose_eval=50, 
                   show_stdv=True
                  )

num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
xgb_model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

[0]	train-rmse:12.6399+0.154977	test-rmse:12.6383+0.309394
[50]	train-rmse:11.0903+0.173162	test-rmse:11.1515+0.321941
[100]	train-rmse:10.0181+0.193437	test-rmse:10.1468+0.339326
[150]	train-rmse:9.28968+0.212088	test-rmse:9.48773+0.359611
[200]	train-rmse:8.80275+0.226347	test-rmse:9.06668+0.37999
[250]	train-rmse:8.47847+0.236952	test-rmse:8.80237+0.39653
[300]	train-rmse:8.26052+0.243699	test-rmse:8.63721+0.408339
[350]	train-rmse:8.09265+0.240265	test-rmse:8.53675+0.417034
[400]	train-rmse:7.94872+0.223081	test-rmse:8.47722+0.423239
[450]	train-rmse:7.83237+0.208945	test-rmse:8.44253+0.428947
[500]	train-rmse:7.72702+0.191756	test-rmse:8.4231+0.431914
[550]	train-rmse:7.63446+0.174185	test-rmse:8.41282+0.433279
[600]	train-rmse:7.55366+0.156574	test-rmse:8.40588+0.433408
[649]	train-rmse:7.48377+0.14259	test-rmse:8.40391+0.43269
650


In [24]:
# check f2-score (to get higher score - increase num_boost_round in previous cell)
from sklearn.metrics import r2_score

# now fixed, correct calculation
print(r2_score(dtrain.get_label(), xgb_model.predict(dtrain)))

0.63188978762


In [91]:
xgb_model.save_model('xgb_{}.model'.format(num_boost_rounds))

In [86]:
# make predictions and save results
y_pred = xgb_model.predict(dtest)
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': y_pred})
output.to_csv('xgboost-boost_rounds{}-pca-ica.csv'.format(num_boost_rounds), index=False)

# Trying to ensemble the results of the two

In [26]:
xgb_train_preds = xgb_model.predict(dtrain)
keras_train_preds = estimator.predict(X)

In [27]:
xgb_test_preds = xgb_model.predict(dtest)
keras_test_preds = estimator.predict(X_test.values).ravel()

In [35]:
cum_train_preds = np.column_stack((keras_train_preds, xgb_train_preds))

In [36]:
cum_test_preds = np.column_stack((keras_test_preds, xgb_test_preds))

In [50]:
# prepare dict of params for xgboost to run with
xgb_params_new = {
    'n_trees': 500, 
    'eta': 0.005,
    'max_depth': 4,
    'subsample': 0.95,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

# form DMatrices for Xgboost training
dtrain_new = xgb.DMatrix(cum_train_preds, y_train)
dtest_new = xgb.DMatrix(cum_test_preds)

# xgboost, cross-validation
cv_result_new = xgb.cv(xgb_params_new, 
                       dtrain, 
                       num_boost_round=500, # increase to have better results (~700)
                       early_stopping_rounds=50,
                       verbose_eval=50, 
                       show_stdv=True
                      )

num_boost_rounds_new = len(cv_result_new)
print(num_boost_rounds_new)

# train model
xgb_model_new = xgb.train(dict(xgb_params_new, silent=0), dtrain_new, num_boost_round=num_boost_rounds_new)

[0]	train-rmse:12.6399+0.154977	test-rmse:12.6383+0.309394
[50]	train-rmse:11.0903+0.173162	test-rmse:11.1515+0.321941
[100]	train-rmse:10.0181+0.193437	test-rmse:10.1468+0.339326
[150]	train-rmse:9.28968+0.212088	test-rmse:9.48773+0.359611
[200]	train-rmse:8.80275+0.226347	test-rmse:9.06668+0.37999
[250]	train-rmse:8.47847+0.236952	test-rmse:8.80237+0.39653
[300]	train-rmse:8.26052+0.243699	test-rmse:8.63721+0.408339
[350]	train-rmse:8.09265+0.240265	test-rmse:8.53675+0.417034
[400]	train-rmse:7.94872+0.223081	test-rmse:8.47722+0.423239
[450]	train-rmse:7.83237+0.208945	test-rmse:8.44253+0.428947
[500]	train-rmse:7.72702+0.191756	test-rmse:8.4231+0.431914
[550]	train-rmse:7.63446+0.174185	test-rmse:8.41282+0.433279
[600]	train-rmse:7.55366+0.156574	test-rmse:8.40588+0.433408
[650]	train-rmse:7.48207+0.14231	test-rmse:8.40394+0.4326
[699]	train-rmse:7.41426+0.127975	test-rmse:8.40462+0.430224
700


In [51]:
# now fixed, correct calculation
print(r2_score(dtrain_new.get_label(), xgb_model_new.predict(dtrain_new)))

0.714911383705


In [45]:
xgb_model_new.save_model('xgb_{}_ensemble.model'.format(num_boost_rounds_new))

In [49]:
# make predictions and save results
y_pred_new = xgb_model_new.predict(dtest_new)
output_new = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': y_pred_new})
output_new.to_csv('xgboost-boost_rounds{}-pca-ica_ensemble.csv'.format(num_boost_rounds_new), index=False)